In [1]:
import torch
import numpy as np

from tqdm import tqdm
import random

from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(8855)
print(device)

cuda


C:\Users\chris\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lstm = torch.nn.LSTM(10,20,1)
stuff = torch.randn(5,3,10)
output = lstm(stuff)
print(output)

(tensor([[[ 1.7630e-02, -1.0847e-01, -1.5039e-01, -9.7330e-03,  2.0457e-03,
           8.5873e-02, -8.5565e-02,  4.7221e-02, -1.9621e-02, -1.3914e-02,
          -8.8130e-02,  2.4225e-02, -1.6868e-02,  2.7444e-02, -3.7166e-02,
           5.5170e-02,  1.4409e-01,  1.3244e-01,  1.0878e-01,  2.8372e-02],
         [-7.7712e-02, -8.6543e-02, -3.5389e-02,  1.5973e-02, -2.7933e-03,
           9.0377e-02, -6.2036e-02,  5.2811e-02, -6.2467e-02, -6.9249e-02,
           1.8516e-03,  1.1086e-01, -4.1779e-02, -1.3126e-01, -3.4962e-02,
          -4.7682e-03,  8.7910e-02, -5.0262e-02, -2.5815e-02, -6.2216e-02],
         [ 1.4144e-01, -6.1399e-03, -1.3166e-01,  7.1492e-02,  1.1627e-01,
           8.5660e-03, -7.2213e-02,  2.0786e-02, -2.3399e-02, -5.9867e-02,
          -5.1596e-03,  1.9993e-01,  4.3648e-03,  8.0428e-02,  3.9716e-02,
           1.4804e-01,  8.1765e-02, -1.1661e-01, -1.0176e-01,  1.0283e-01]],

        [[-4.0178e-02, -1.4586e-01, -5.1796e-02, -1.1073e-02,  3.8176e-02,
           2.1209e-

In [3]:
target_classes = ['O','B-geo','B-gpe','B-per','I-geo','B-org','I-org','B-tim','B-art','I-art','I-per','I-gpe','I-tim','B-nat','B-eve','I-eve','I-nat']

class LSTMClassifier(torch.nn.Module):
    def __init__(self, embedding, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding)
        self.lstm = torch.nn.LSTM(input_size=embedding, hidden_size=hidden_dim, num_layers=1, batch_first=True, bidirectional=True)
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim*2, hidden_dim),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim, tagset_size)
        )
        self.soft = torch.nn.Softmax(dim=2)
        
    def forward(self, X_batch):
        X_batch = X_batch.to(device)
        embeddings = self.embedding_layer(X_batch)
        embeddings = embeddings.float().requires_grad_()
        output, _ = self.lstm(embeddings)
        out = self.dense(output)
        soft = self.soft(out)
        return soft
        
lstm_classifier = LSTMClassifier(128, 75, 35181, len(target_classes)).to(device)

input = torch.tensor([[  345,   134,  1470,   363,    31,     9,    16,  7202,  2280,    93,  4268,   172,
    35,     9,  9063,     1,  3243,    78, 13429,  3123,    21,    35, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,
 35180, 35180, 35180, 35180, 35180, 35180, 35180, 35180,]])
print(input.shape)
out = lstm_classifier(input)
print(out, out.shape)
print(out[0][0])

torch.Size([1, 128])
tensor([[[0.0618, 0.0551, 0.0568,  ..., 0.0668, 0.0621, 0.0539],
         [0.0612, 0.0563, 0.0558,  ..., 0.0650, 0.0629, 0.0571],
         [0.0602, 0.0527, 0.0576,  ..., 0.0621, 0.0654, 0.0583],
         ...,
         [0.0615, 0.0515, 0.0595,  ..., 0.0639, 0.0626, 0.0539],
         [0.0610, 0.0515, 0.0595,  ..., 0.0641, 0.0627, 0.0540],
         [0.0601, 0.0516, 0.0594,  ..., 0.0647, 0.0631, 0.0539]]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>) torch.Size([1, 128, 17])
tensor([0.0618, 0.0551, 0.0568, 0.0578, 0.0551, 0.0655, 0.0542, 0.0567, 0.0605,
        0.0664, 0.0631, 0.0552, 0.0539, 0.0552, 0.0668, 0.0621, 0.0539],
       device='cuda:0', grad_fn=<SelectBackward0>)


In [4]:
def get_vocab(vocab_path, tags_path):
    vocab = {}
    with open(vocab_path, encoding="ISO-8859-1") as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i 
    vocab['<PAD>'] = len(vocab)
    tag_map = {}
    with open(tags_path, encoding="ISO-8859-1") as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i
            
    return vocab, tag_map

def get_params(vocab, tag_map, sentences_file, labels_file):
    sentences = []
    labels = []
    
    with open(sentences_file, encoding="ISO-8859-1") as f:
        for sentence in f.read().splitlines():
            s = [vocab[token] if token in vocab
                 else vocab['UNK']
                 for token in sentence.split(' ')]
            sentences.append(s)
            
    with open(labels_file, encoding="ISO-8859-1") as f:
        for sentence in f.read().splitlines():
            l = [tag_map[label] for label in sentence.split(' ')]
            labels.append(l)
    return sentences, labels, len(sentences)

In [5]:
vocab, tag_map = get_vocab("./words.txt", "./tags.txt")
t_sentences, t_labels, t_size = get_params(vocab, tag_map, './train/sentences.txt', './train/labels.txt')
print(t_sentences[0], t_labels[0], t_size)
test_sentences, test_labels, test_size = get_params(vocab, tag_map, './test/sentences.txt', './test/labels.txt')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 9, 15, 1, 16, 17, 18, 19, 20, 21] [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0] 33570


In [6]:
def data_generator(batch_size, x, y, pad, shuffle=False, verbose=False):
    '''
      Input: 
        batch_size - integer describing the batch size
        x - list containing sentences where words are represented as integers
        y - list containing tags associated with the sentences
        shuffle - Shuffle the data order
        pad - an integer representing a pad character
        verbose - Print information during runtime
      Output:
        a tuple containing 2 elements:
        X - np.ndarray of dim (batch_size, max_len) of padded sentences
        Y - np.ndarray of dim (batch_size, max_len) of tags associated with the sentences in X
    '''
    
    # count the number of lines in data_lines
    num_lines = len(x)
    
    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]
    
    # shuffle the indexes if shuffle is set to True
    if shuffle:
        random.shuffle(lines_index)
    
    index = 0 # tracks current location in x, y
    while True:
        buffer_x = [0] * batch_size # Temporal array to store the raw x data for this batch
        buffer_y = [0] * batch_size # Temporal array to store the raw y data for this batch
                
        
        # Copy into the temporal buffers the sentences in x[index : index + batch_size] 
        # along with their corresponding labels y[index : index + batch_size]
        # Find maximum length of sentences in x[index : index + batch_size] for this batch. 
        # Reset the index if we reach the end of the data set, and shuffle the indexes if needed.
        max_len = 128
        for i in range(batch_size):
             # if the index is greater than or equal to the number of lines in x
            if index >= num_lines:
                # then reset the index to 0
                index = 0
                # re-shuffle the indexes if shuffle is set to True
                if shuffle:
                    random.shuffle(lines_index)
            
            # The current position is obtained using `lines_index[index]`
            # Store the x value at the current position into the buffer_x
            buffer_x[i] = x[lines_index[index]]
            
            # Store the y value at the current position into the buffer_y
            buffer_y[i] = y[lines_index[index]]
            
            lenx = len(buffer_x[i])    #length of current x[]
            if lenx > max_len:
                max_len = lenx                   #max_len tracks longest x[]
            
            # increment index by one
            index += 1


        # create X,Y, NumPy arrays of size (batch_size, max_len) 'full' of pad value
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len, 17), pad)

        # copy values from lists to NumPy arrays. Use the buffered values
        for i in range(batch_size):
            # get the example (sentence as a tensor)
            # in `buffer_x` at the `i` index
            x_i = buffer_x[i]
            
            # similarly, get the example's labels
            # in `buffer_y` at the `i` index
            y_i = buffer_y[i]
            
            # Walk through each word in x_i
            for j in range(len(x_i)):
                # store the word in x_i at position j into X
                X[i, j] = x_i[j]
                
                # store the label in y_i at position j into Y
                Y[i, j] = y_i[j]
                
                # one hot encode 3rd dim
                for k in range(17):
                    if k == y_i[j]:
                        Y[i, j, k] = 1
                    else:
                        Y[i, j, k] = 0       

        if verbose: print("index=", index)
        yield((X,Y))

In [7]:
train_loader = data_generator(64, t_sentences, t_labels, vocab["<PAD>"],True)
test_loader = data_generator(64, test_sentences, test_labels, vocab["<PAD>"],True)

for X,Y in train_loader:
    print(X.shape, Y.shape)
    break

(64, 128) (64, 128, 17)


In [8]:
def Compare(model, loss_fn, test_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [], [], []
        its = 0
        for X, Y in tqdm(test_loader):
            X = torch.Tensor(X).to(device).long()
            Y = torch.FloatTensor(Y).to(device)
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            
            Y_shuffled.append(Y)
            Y_preds.append(preds)
            its += 1
            if its > 7194:
                break
            
        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)
        
        print(f"Validation Loss: {torch.tensor(losses).mean():.3f}")
        mask = Y_shuffled.detach().cpu().numpy() != vocab["<PAD>"]
        
        acc = np.sum((Y_preds.detach().cpu().numpy() == Y_shuffled.detach().cpu().numpy()) * mask) / np.sum(mask)
        print(f"Validation Accuracy: {acc}")
            
def TrainingLoop(model, loss_fn, optimizer, train_loader, test_loader, epochs=10):
    for i in range(1, epochs+1):
        print(f"Epoch: {i}")
        losses = []
        its = 0
        for X, Y in tqdm(train_loader):
            model.zero_grad()
            
            X = torch.Tensor(X).to(device).long()
            Y = torch.FloatTensor(Y).to(device)
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            
            loss.backward()
            optimizer.step()
            
            its += 1
            if its > 33570:
                break
            if its % 1000 == 0:
                clear_output(wait=True)
                print(f"Train Loss: {torch.tensor(losses).mean():.3f}")
        Compare(model, loss_fn, test_loader)
        torch.cuda.empty_cache()

In [9]:
epoch=1
learning_rate = 1e-3

loss_fn = torch.nn.CrossEntropyLoss()
lstm_classifier = LSTMClassifier(128, 75, 35181, 17).to(device)
optimizer = torch.optim.SGD(lstm_classifier.parameters(), lr=learning_rate)

In [10]:
TrainingLoop(lstm_classifier, loss_fn, optimizer, train_loader, test_loader, epoch)


33018it [04:06, 135.53it/s]

Train Loss: 18114870.000


33570it [04:10, 133.79it/s]
7194it [00:40, 175.72it/s]


Validation Loss: 18141840.000
Validation Accuracy: 0.0


In [12]:
torch.save(lstm_classifier, "./LSTM_NER.pth")
model = torch.load("./LSTM_NER.pth")

In [13]:
def predict(sentence, model, vocab, tag_map):
    s = [vocab[token] if token in vocab else vocab['UNK'] for token in sentence.split(' ')]
    batch_data = np.ones((1, len(s)))
    batch_data[0][:] = s
    sentence = torch.Tensor(batch_data).to(device).long()
    output = model(sentence)
    outputs = output.argmax(dim=2)
    labels = list(tag_map.keys())
    pred = []
    for i in range(len(outputs[0])):
        idx = outputs[0][i] 
        pred_label = labels[idx]
        pred.append(pred_label)
    return pred
    
sentence = "Peter Navarro, the White House director of trade and manufacturing policy of U.S, said in an interview on Sunday morning that the White House was working to prepare for the possibility of a second wave of the coronavirus in the fall, though he said it wouldn't necessarily come"
s = [vocab[token] if token in vocab else vocab['UNK'] for token in sentence.split(' ')]
predictions = predict(sentence, lstm_classifier, vocab, tag_map)
for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x,y)

Peter B-tim
Navarro, B-tim
the B-tim
White B-tim
House B-tim
director B-tim
of B-tim
trade B-tim
and B-tim
manufacturing B-tim
policy B-tim
of B-tim
U.S, B-tim
said B-tim
in B-tim
an B-tim
interview B-tim
on B-tim
Sunday B-tim
morning B-tim
that B-tim
the B-tim
White B-tim
House B-tim
was B-tim
working B-tim
to B-tim
prepare B-tim
for B-tim
the B-tim
possibility B-tim
of B-tim
a B-tim
second B-tim
wave B-tim
of B-tim
the B-tim
coronavirus B-tim
in B-tim
the B-tim
fall, B-tim
though B-tim
he B-tim
said B-tim
it B-tim
wouldn't B-tim
necessarily B-tim
come B-tim
